
# ANALYSIS <!-- testing a lot of stuff -->

In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
file_path = '../airflow/data/production/flights.csv'
df = pd.read_csv(file_path)
df.head()

In [ ]:
df.info()

In [ ]:
# Group by airline and calculate mean delays
airline_delays = df.groupby('airline')['delay_in_minutes'].mean().sort_values(ascending=False)

# Plot the results
plt.figure(figsize=(12, 6))
sns.barplot(x=airline_delays.index, y=airline_delays.values, palette='viridis')
plt.title('Average Arrival Delays by Airline')
plt.xlabel('Airline')
plt.ylabel('Average Delay (Minutes)')
plt.xticks(rotation=90)
plt.show()

In [68]:
import re

# Function to determine if the wind is strong (e.g., > 30 knots)
def is_strong_wind(wind):
    match = re.search(r'(\d{2})', wind)  # Find the first two digits (wind speed in knots)
    if match:
        wind_speed = int(match.group(1))
        return wind_speed > 30
    return False

# Function to check visibility (e.g., < 3 statute miles)
def is_low_visibility(visibility):
    if 'SM' in visibility:
        visibility_value = float(visibility.replace('SM', ''))
        return visibility_value < 3
    return False

# Function to check if sky condition is bad (BKN, OVC, TCU, CB)
def is_bad_sky_condition(sky_condition):
    if any(condition in sky_condition for condition in ['BKN', 'OVC', 'TCU', 'CB']):
        return True
    return False


# Function to check if the temperature is extreme (e.g., below -5°C or above 40°C)
def is_extreme_temperature(temperature):
    match = re.match(r'(\d+)', temperature.split('/')[0])  # Extract temperature from format like "07/02"
    if match:
        temp = int(match.group(1))
        return temp < -5 or temp > 40
    return False

# Now we apply these functions to create a 'BadWeatherDeparture' column
def determine_bad_weather_departure(weather_row):
    wind, visibility, sky_condition, temperature = weather_row
    if (is_strong_wind(wind) or 
        is_low_visibility(visibility) or 
        is_bad_sky_condition(sky_condition) or 
        is_extreme_temperature(temperature)):
        return 1  # Bad weather
    return 0  # Good weather


In [ ]:
df_departure = df

# Apply the function to the 'WeatherDeparture' column
df_departure['BadWeatherDeparture'] = df_departure[['departure_wind', 'departure_visibility', 'departure_sky_condition', 'departure_temperature']].apply(determine_bad_weather_departure, axis=1)

df_departure = df_departure.dropna(subset=['delay_in_minutes', 'BadWeatherDeparture'])

# Create a scatter plot to show the correlation
plt.figure(figsize=(10, 6))
sns.boxplot(x='BadWeatherDeparture', y='delay_in_minutes', data=df_departure, palette='coolwarm')

# Set plot title and labels
plt.title('Correlation Between Departure Delay and Bad Weather Conditions')
plt.xlabel('Bad Weather Departure (0 = No, 1 = Yes)')
plt.ylabel('Departure Delay (in minutes)')

# Show plot
plt.show()

In [ ]:
df_destination = df

# Apply the function to the 'WeatherDeparture' column
df_destination['BadWeatherDestination'] = df_destination[['destination_wind', 'destination_visibility', 'departure_sky_condition', 'departure_temperature']].apply(determine_bad_weather_departure, axis=1)

df_destination = df_destination.dropna(subset=['delay_in_minutes', 'BadWeatherDestination'])

# Create a scatter plot to show the correlation
plt.figure(figsize=(10, 6))
sns.boxplot(x='BadWeatherDestination', y='delay_in_minutes', data=df_destination, palette='coolwarm')

# Set plot title and labels
plt.title('Correlation Between Destination Delay and Bad Weather Conditions')
plt.xlabel('Bad Weather Destination (0 = No, 1 = Yes)')
plt.ylabel('Destination Delay (in minutes)')

# Show plot
plt.show()

In [ ]:
# Create a new column for BadWeather at either departure or destination (logical OR)
df['BadWeatherEither'] = df['BadWeatherDeparture'] | df['BadWeatherDestination']

In [ ]:
# Analyze by time of the day

# Extract hour from the scheduled_time
df['hour'] = pd.to_datetime(df['scheduled_time']).dt.hour

# Create time periods
def get_time_period(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    else:
        return 'Night'

df['time_period'] = df['hour'].apply(get_time_period)

# Group by time period and calculate average delay
time_period_delays = df.groupby('time_period')['delay_in_minutes'].mean()

# Plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x=time_period_delays.index, y=time_period_delays.values, palette='coolwarm')
plt.title('Average Delay by Time of Day')
plt.xlabel('Time Period')
plt.ylabel('Average Delay (Minutes)')
plt.show()


In [ ]:
# which weather factor has the most impact?

# Create separate columns for each weather condition
df['StrongWindDeparture'] = df['departure_wind'].apply(is_strong_wind)
df['LowVisibilityDeparture'] = df['departure_visibility'].apply(is_low_visibility)
df['BadSkyConditionDeparture'] = df['departure_sky_condition'].apply(is_bad_sky_condition)
df['ExtremeTemperatureDeparture'] = df['departure_temperature'].apply(is_extreme_temperature)

# Calculate average delays for each weather condition
weather_impact = df.groupby(['StrongWindDeparture', 'LowVisibilityDeparture', 'BadSkyConditionDeparture', 'ExtremeTemperatureDeparture'])['delay_in_minutes'].mean()

# Print the result
print(weather_impact)


In [ ]:
# Check the Correlation Between Departure and Destination Weather

# Create a new column for bad weather at both ends
df['BadWeatherBothEnds'] = (df['BadWeatherDeparture'] & df['BadWeatherDestination']).astype(int)

# Group by BadWeatherBothEnds and calculate average delay
both_ends_delay = df.groupby('BadWeatherBothEnds')['delay_in_minutes'].mean()

# Plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x=both_ends_delay.index, y=both_ends_delay.values, palette='coolwarm')
plt.title('Average Delay for Bad Weather at Both Ends')
plt.xlabel('Bad Weather at Both Ends (0 = No, 1 = Yes)')
plt.ylabel('Average Delay (Minutes)')
plt.show()

In [74]:
# Create separate columns for each weather condition at the destination
df['StrongWindDestination'] = df['destination_wind'].apply(is_strong_wind)
df['LowVisibilityDestination'] = df['destination_visibility'].apply(is_low_visibility)
df['BadSkyConditionDestination'] = df['destination_sky_condition'].apply(is_bad_sky_condition)
df['ExtremeTemperatureDestination'] = df['destination_temperature'].apply(is_extreme_temperature)

In [ ]:
# Compare Short Delays vs. Long Delays (Threshold Analysis)

# Create a new column for delay categories
def categorize_delay(delay):
    if delay <= 15:
        return 'Short Delay'
    elif 15 < delay <= 30:
        return 'Moderate Delay'
    else:
        return 'Long Delay'

df['delay_category'] = df['delay_in_minutes'].apply(categorize_delay)

# Group by delay category and calculate average weather conditions
delay_weather = df.groupby('delay_category')[['StrongWindDeparture', 'LowVisibilityDeparture', 'BadSkyConditionDeparture', 'ExtremeTemperatureDeparture']].mean()

# Print the result
print(delay_weather)



In [76]:
# Create separate columns for each weather condition at the destination
df['StrongWindDestination'] = df['destination_wind'].apply(is_strong_wind)
df['LowVisibilityDestination'] = df['destination_visibility'].apply(is_low_visibility)
df['BadSkyConditionDestination'] = df['destination_sky_condition'].apply(is_bad_sky_condition)
df['ExtremeTemperatureDestination'] = df['destination_temperature'].apply(is_extreme_temperature)


In [ ]:
# Count how often each extreme weather condition is true for both departure and destination
weather_counts = df[['StrongWindDeparture', 'LowVisibilityDeparture', 'BadSkyConditionDeparture', 'ExtremeTemperatureDeparture', 'StrongWindDestination', 'LowVisibilityDestination', 'BadSkyConditionDestination', 'ExtremeTemperatureDestination']].sum()

# Display the counts
print(weather_counts)


In [ ]:
# Convert 'scheduled_time' to datetime format
df['scheduled_time'] = pd.to_datetime(df['scheduled_time'])

# Extract the date
df['date'] = df['scheduled_time'].dt.date

# Group by date and calculate average delay
delays_by_date = df.groupby('date')['delay_in_minutes'].mean()

# Plot the delays by date
plt.figure(figsize=(12, 6))
sns.lineplot(x=delays_by_date.index, y=delays_by_date.values, marker='o', color='b')
plt.title('Average Flight Delay by Date')
plt.xlabel('Date')
plt.ylabel('Average Delay (Minutes)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Classify delays into categories by date
def classify_delay(delay):
    if delay <= 15:
        return 'Short Delay'
    elif 15 < delay <= 30:
        return 'Moderate Delay'
    else:
        return 'Long Delay'

# Apply the classification to create a new column
df['delay_category'] = df['delay_in_minutes'].apply(classify_delay)

# Group by date and delay category
delays_by_date_category = df.groupby(['date', 'delay_category']).size().unstack(fill_value=0)

# Plot the results with intuitive colors
delays_by_date_category.plot(kind='bar', stacked=True, figsize=(15, 8), 
                             color=['red', 'yellow', 'green'])
plt.title('Flight Delay Classification by Date')
plt.xlabel('Date')
plt.ylabel('Number of Flights')
plt.xticks(rotation=45)
plt.legend(title='Delay Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
# Filter out rows with delays
df_departure = df.dropna(subset=['delay_in_minutes', 'BadWeatherDeparture'])

# Convert delay_in_minutes to a binary flag (1 if delay > 0, 0 otherwise)
df_departure['HasDelay'] = df_departure['delay_in_minutes'].apply(lambda x: 1 if x > 0 else 0)

# Group by bad weather condition and calculate the total number of flights and delays
flight_counts = df_departure.groupby('BadWeatherDeparture')['HasDelay'].count().reset_index(name='TotalFlights')
delay_counts = df_departure.groupby('BadWeatherDeparture')['HasDelay'].sum().reset_index(name='Delays')

# Merge the two DataFrames
merged_counts = pd.merge(delay_counts, flight_counts, on='BadWeatherDeparture')

# Calculate the normalized delay rate
merged_counts['NormalizedDelayRate'] = merged_counts['Delays'] / merged_counts['TotalFlights']

# Create a bar plot to show the normalized correlation
plt.figure(figsize=(10, 6))
sns.barplot(x='BadWeatherDeparture', y='NormalizedDelayRate', data=merged_counts, palette='coolwarm')

# Set plot title and labels
plt.title('Normalized Correlation Between Departure Delay Rate and Bad Weather Conditions')
plt.xlabel('Bad Weather Departure (0 = No, 1 = Yes)')
plt.ylabel('Normalized Delay Rate')

# Show plot
plt.show()


In [ ]:
# Create a DataFrame that shows delays and bad weather conditions at departure and destination
delays_weather_df = df[['scheduled_time', 'departure_icao', 'arrival_icao', 'delay_in_minutes', 'BadWeatherDeparture', 'BadWeatherDestination']]

# Rename columns for clarity
delays_weather_df = delays_weather_df.rename(columns={
    'BadWeatherDeparture': 'BadWeather_Departure',
    'BadWeatherDestination': 'BadWeather_Destination'
})

# Display the table
import pandas as pd
pd.set_option('display.max_rows', None)  # Show all rows (adjust as needed)
print(delays_weather_df)

# Alternatively, display the table as a sample output
delays_weather_df.head(10)


In [ ]:
# Check the Correlation Between Departure and Destination Weather

# Create a new column for bad weather at both ends
df['BadWeatherBothEnds'] = (df['BadWeatherDeparture'] & df['BadWeatherDestination']).astype(int)

# Group by BadWeatherBothEnds and calculate average delay
both_ends_delay = df.groupby('BadWeatherBothEnds')['delay_in_minutes'].mean()

# Plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x=both_ends_delay.index, y=both_ends_delay.values, palette='coolwarm')
plt.title('Average Delay for Bad Weather at Both Ends')
plt.xlabel('Bad Weather at Both Ends (0 = No, 1 = Yes)')
plt.ylabel('Average Delay (Minutes)')
plt.show()


In [ ]:
# Group by BadWeatherEither and calculate the average delay
either_ends_delay = df.groupby('BadWeatherEither')['delay_in_minutes'].mean()

# Plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x=either_ends_delay.index, y=either_ends_delay.values, palette='coolwarm')
plt.title('Average Delay for Bad Weather at Either Departure or Destination')
plt.xlabel('Bad Weather at Either End (0 = No, 1 = Yes)')
plt.ylabel('Average Delay (Minutes)')
plt.show()


In [ ]:
# Group by airline and BadWeatherEither to calculate the mean delay
airline_weather_impact = df.groupby(['airline', 'BadWeatherEither'])['delay_in_minutes'].mean().unstack()

# Display the DataFrame
print(airline_weather_impact)

# Plot the impact of bad weather on delays for each airline
plt.figure(figsize=(14, 8))
airline_weather_impact.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='coolwarm')
plt.title('Average Delay by Airline under Bad Weather Conditions (Departure or Destination)')
plt.xlabel('Airline')
plt.ylabel('Average Delay (Minutes)')
plt.xticks(rotation=45)
plt.legend(['Good Weather (0)', 'Bad Weather (1)'])
plt.show()



In [ ]:
# --- Plot 1: Average Delay by Date for Good Weather ---
good_weather = df[df['BadWeatherEither'] == 0]
delays_by_date_good = good_weather.groupby('date')['delay_in_minutes'].mean()

plt.figure(figsize=(12, 6))
sns.lineplot(x=delays_by_date_good.index, y=delays_by_date_good.values, marker='o', color='g')
plt.title('Average Flight Delay by Date (Good Weather)')
plt.xlabel('Date')
plt.ylabel('Average Delay (Minutes)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# --- Plot 2: Average Delay by Date for Bad Weather ---
bad_weather = df[df['BadWeatherEither'] == 1]
delays_by_date_bad = bad_weather.groupby('date')['delay_in_minutes'].mean()

plt.figure(figsize=(12, 6))
sns.lineplot(x=delays_by_date_bad.index, y=delays_by_date_bad.values, marker='o', color='r')
plt.title('Average Flight Delay by Date (Bad Weather)')
plt.xlabel('Date')
plt.ylabel('Average Delay (Minutes)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# --- Plot 3: Comparison of Average Delays (Good Weather vs. Bad Weather) ---
plt.figure(figsize=(12, 6))
sns.lineplot(x=delays_by_date_good.index, y=delays_by_date_good.values, marker='o', label='Good Weather', color='g')
sns.lineplot(x=delays_by_date_bad.index, y=delays_by_date_bad.values, marker='o', label='Bad Weather', color='r')
plt.title('Comparison of Average Flight Delay by Date (Good vs. Bad Weather)')
plt.xlabel('Date')
plt.ylabel('Average Delay (Minutes)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()